## Collect House and Senate election outcomes from wikipedia
This notbook uses the wikitables python library to grab election data from wikipedia.
Election outcomes are then matched with FEC funding data

In [181]:
def importHouseData(wikipage,start,stop):
    from wikitables import import_tables
    import re
    import pandas as pd
    tables = import_tables(wikipage)
    States = ['Alabama','Alaska','Arizona' ,'Arkansas' ,'California' ,'Colorado','Connecticut',
              'Delaware' ,'Florida' ,'Georgia' ,'Hawaii' ,'Idaho' ,'Illinois' ,'Indiana','Iowa',
              'Kansas' ,'Kentucky' ,'Louisiana' ,'Maine','Maryland' ,'Massachusetts' ,'Michigan',
              'Minnesota' ,'Mississippi' ,'Missouri','Montana' ,'Nebraska','Nevada' ,
              'New Hampshire' ,'New Jersey' ,'New Mexico' ,'New York' ,'North Carolina' ,
              'North Dakota' ,'Ohio' ,'Oklahoma' ,'Oregon' ,'Pennsylvania' ,'Rhode Island' ,
              'South Carolina' ,'South Dakota' ,'Tennessee' ,'Texas' ,'Utah' ,'Vermont' ,
              'Virginia' ,'Washington' ,'West Virginia' ,'Wisconsin' ,'Wyoming']
    df = pd.DataFrame(columns=['State','District','Candidate','Vote%','Party'])
    ## parse the table
    for table in tables[start:stop]:
        ## get state from first row
        currState = 'FAILED'
        for state in States:
            if state in table.rows[0]['District'].value:
                currState = state
                States.remove(state)

        ## now parse candidates, parties, %vote
        district = 1
        for row in table.rows:
                #get line to be parsed
                fullString = '{Representative}'.format(**row)
                # regex out candidates
                candidates = filter(None,re.split('\xe2\x88\x9a |%|\d+\.\d+|<\d|\([A-Za-z0-9_/\- \.]+\) |\d|',fullString))
                if ' ' in candidates:
                    candidates.remove(' ')

                # regex out parties
                parties = re.findall('\([A-Za-z0-9_/\- \.]+\)',fullString)
                for p in parties: # make sure that wiki clarifications don't go in parties field
                    if 'politician' in p:
                        parties.remove(p)
                # regex %vote
                percentVotes = re.findall('\d+\.\d+|(?=<)\d|[0-9]+(?=%)',fullString)

                for cand,perc,part in map(None,candidates,percentVotes,parties):
                    if cand: #more parsing to remove wiki clarifications
                        cand = re.sub('\([\wA-Za-z. ]+\)', '', cand)
                    if part: # remove parenthesis on party
                        part = re.sub(r'\(|\)', '', part)
                    df.loc[len(df)] = [currState,district,cand,perc,part]
                    
                district+=1
    return df



In [182]:
house = importHouseData('United States House of Representatives elections, 2016',5,55)

In [183]:
def importSenateData(wikipage,start,stop):
    from wikitables import import_tables
    import re
    import pandas as pd
    tables = import_tables('United States Senate elections, 2016')
    States = ['Alabama','Alaska','Arizona' ,'Arkansas' ,'California' ,'Colorado','Connecticut',
              'Delaware' ,'Florida' ,'Georgia' ,'Hawaii' ,'Idaho' ,'Illinois' ,'Indiana','Iowa',
              'Kansas' ,'Kentucky' ,'Louisiana' ,'Maine','Maryland' ,'Massachusetts' ,'Michigan',
              'Minnesota' ,'Mississippi' ,'Missouri','Montana' ,'Nebraska','Nevada' ,
              'New Hampshire' ,'New Jersey' ,'New Mexico' ,'New York' ,'North Carolina' ,
              'North Dakota' ,'Ohio' ,'Oklahoma' ,'Oregon' ,'Pennsylvania' ,'Rhode Island' ,
              'South Carolina' ,'South Dakota' ,'Tennessee' ,'Texas' ,'Utah' ,'Vermont' ,
              'Virginia' ,'Washington' ,'West Virginia' ,'Wisconsin' ,'Wyoming']
    df = pd.DataFrame(columns=['State','District','Candidate','Vote%','Party'])
    ## parse the table
    for table in tables[start:stop]:
        ## now parse candidates, parties, %vote
        district = 0
        for row in table.rows:
            ## get state from first row
            currState = 'FAILED'
            for state in States:
                if state in row['Status'].value:
                    currState = state
                    States.remove(state)
                    #get line to be parsed
            fullString = '{Senator}'.format(**row)
            # regex out candidates
            candidates = filter(None,re.split('\xe2\x88\x9a |%|\d+\.\d+|<\d|\([A-Za-z0-9_/\- \.]+\) |\d|',fullString))
            if ' ' in candidates:
                candidates.remove(' ')

            # regex out parties
            parties = re.findall('\([A-Za-z0-9_/\- \.]+\)',fullString)
            for p in parties:
                if 'politician' in p:
                    parties.remove(p)
            # regex %vote
            percentVotes = re.findall('\d+\.\d+|(?=<)\d|[0-9]+(?=%)',fullString)

            for cand,perc,part in map(None,candidates,percentVotes,parties):
                if cand:
                        cand = re.sub('\([\wA-Za-z. ]+\)', '', cand)
                if part:
                    part = re.sub(r'\(|\)', '', part)
                df.loc[len(df)] = [currState,district,cand,perc,part]
                
        return df

In [149]:
senate = importSenateData('United States House of Representatives elections, 2016',8,9)
congress = pd.merge(senate,house,how='outer')
## noticed problem parsing arizona
congress.set_value(9,'Vote%',5)
congress.set_value(8,'Vote%',None)

,State,District,Candidate,Vote%,Party
0,Alabama,0.0,Richard Shelby,64.2,Republican
1,Alabama,0.0,Ron Crumpton,35.8,Democratic
2,Alaska,0.0,Lisa Murkowski,44,Republican
3,Alaska,0.0,Joe Miller,29,Libertarian
4,Alaska,0.0,Margaret Stock,13,Independent
5,Alaska,0.0,Ray Metcalfe,12,Democratic
6,Arizona,0.0,John McCain,53,Republican
7,Arizona,0.0,Ann Kirkpatrick,41,Democratic
8,Arizona,0.0,Pat Quinn,None,independent/Write-in
9,Arizona,0.0,Gary Swing,5,Green/Write-in


In [184]:
import pandas as pd
# this csv was acquired here http://www.fec.gov/data/CandidateSummary.do?format=html
# The resulting data is loaded into a google doc for collaborators
googleDoc = pd.read_csv('ElectionOutcomes.csv')
googleDoc.head()

,ID,Candidate,Party,State Code,State,District,Outcome,Senate? (0/1),Checked? (0/1)
0,H6AK00045,"YOUNG, DONALD E",REP,AK,Alaska,0,50.3,0.0,0
1,S8AK00082,"METCALFE, RAY HOMER",DEM,AK,Alaska,0,NaN,NaN,0
2,S4AK00099,"MURKOWSKI, LISA",REP,AK,Alaska,0,44.0,1.0,0
3,S6AK00144,"LOCHNER, BOB",REP,AK,Alaska,0,NaN,NaN,0
4,S6AK00151,"STOCK, MARGARET",IND,AK,Alaska,0,NaN,NaN,0


In [185]:
import re
import numpy as np
## I want to match candidates based on state, then candidate name
States = ['Alabama','Alaska','Arizona' ,'Arkansas' ,'California' ,'Colorado','Connecticut',
              'Delaware' ,'Florida' ,'Georgia' ,'Hawaii' ,'Idaho' ,'Illinois' ,'Indiana','Iowa',
              'Kansas' ,'Kentucky' ,'Louisiana' ,'Maine','Maryland' ,'Massachusetts' ,'Michigan',
              'Minnesota' ,'Mississippi' ,'Missouri','Montana' ,'Nebraska','Nevada' ,
              'New Hampshire' ,'New Jersey' ,'New Mexico' ,'New York' ,'North Carolina' ,
              'North Dakota' ,'Ohio' ,'Oklahoma' ,'Oregon' ,'Pennsylvania' ,'Rhode Island' ,
              'South Carolina' ,'South Dakota' ,'Tennessee' ,'Texas' ,'Utah' ,'Vermont' ,
              'Virginia' ,'Washington' ,'West Virginia' ,'Wisconsin' ,'Wyoming']

for state in States:
    # select only current state
    distFund = googleDoc.loc[funding['State']==state]
    distCong = congress.loc[congress['State']==state]
    for i,cong in distCong.iterrows():
        # for each congressional candidate ...
        nameList = [x.upper() for x in filter(None,re.split(' ',cong['Candidate']))]
        counter = [0]*len(distFund)
        iFund=0
        # attempt to match name to fundraisers ...
        for i,fund in distFund.iterrows():
            for name in nameList:
                if name in fund['Candidate']:
                    counter[iFund]+=1
            iFund+=1
        # if "strong" match: update the fundraising df to reflect election outcome
        if sum([int(c==max(counter)) for c in counter])==1:
            iFund = distFund.index.values[np.argmax(counter)]
            googleDoc.set_value(iFund,'Outcome', cong['Vote%'])
            
googleDoc.to_csv('googleDoc.csv')